In [69]:
from selenium import webdriver
import time
import datetime
import math
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import openpyxl
import pandas as pd
import warnings

In [70]:
def charms (i):
    name = str(df.iloc[i]['name'])
    if "крест" in name or "христиан" in name or "икона" in name or "мусульман" in name or "спаси" in name or "сохрани" in name or "месяц" in name or "аллах" in name or "пророк" in name or "будда" in name or "отче" in name or "молитва" in name or "икона" in name or "лунниц" in name or "ангел" in name or "эрцгамм" in name or "чудо" in name or "фатим" in name or "иисус" in name or "б.м" in name or "давид" in name or "матрон" in name or "госп" in name or "хамс" in name:
        return "КУЛЬТ"
    else:
        return "ДЕКОР"

In [71]:
def bracs (link):
    url =f'https://card.wb.ru/cards/v2/detail?appType=1&curr=rub&dest=-1257786&spp=30&nm={link}'
    sizes = requests.get(url, verify=False, timeout=30).json()
    try:
        r = sizes['data']['products'][0]['sizes'][0]['name']
        if r.rfind("-") > -1:
            wei = r[r.rfind("-")+1:].strip().lower().replace(",",".").replace("гр.","").replace("гр","")
        elif r.rfind("~") > -1:
            wei = r[r.rfind("~")+1:].strip().lower().replace(",", ".").replace("гр.","").replace("гр","")
        elif r.rfind(" ") > -1:
            wei = r[r.rfind(" ")+1:].strip().lower().replace(",", ".").replace("гр.","").replace("гр","")
        wei = float(wei)
        if wei > 0 and wei < 15:
            return wei
    except:
        return np.nan

In [62]:
def card (art):
    group = np.nan
    name = np.nan
    weight = np.nan
    lock_type = np.nan
    metal = np.nan
    insert = np.nan
    competitor = np.nan
    date = datetime.date.today()
    i = 0
    if int(art)//100000>spis[-1]:
        i = len(spis)
    else:
        while int(art)//100000 > spis[i]:
            i += 1
    i += 1
    if i < 10:
        i = "0"+str(i)
    current_link = f'https://basket-{i}.wbbasket.ru/vol{int(art)//100000}/part{int(art)//1000}/{art}/info/ru/card.json'
    try:
        basket = requests.get(current_link, verify=False, timeout=30).json()
    except:
        time.sleep(60)
        basket = requests.get(current_link, verify=False, timeout=30).json()
    try:
        competitor = basket['selling']['brand_name']
    except:
        pass
    try:
        name = basket['imt_name'].lower()
    except:
        pass
    try:
        group = basket['subj_name']
    except:
        pass
    try:
        for option in basket['options']:
            if option['name'] == 'Минимальный вес (г)':
                weight = float(option['value'].replace('г',''))
            elif option['name'] == 'Вставка':
                insert = option['value']
            elif option['name'] == 'Вид замка':
                lock_type = option['value']
            elif option['name'] == 'Состав ювелирного изделия':
                metal = option['value']
    except:
        pass
    try:
        for option in basket['grouped_options'][0]['options']:
            if option['name'] == 'Минимальный вес (г)':
                weight = float(option['value'].replace('г',''))
            elif option['name'] == 'Вставка':
                insert = option['value']
            elif option['name'] == 'Вид замка':
                lock_type = option['value']
            elif option['name'] == 'Состав ювелирного изделия':
                metal = option['value'].lower()
    except:
        pass
    try:
        if np.isnan(weight) and group in ["ЦБ БРАСЛЕТЫ","ЦБ ЦЕПИ"]:
            weight = float(bracs(art))
    except:
        pass
    try:
        if group == "ЦБ ЦЕПИ":
            for option in basket['options']:
                if 'плетен' in option['name']:
                    lock_type = option['value']
    except:
        pass
    try:
        if group == "ЦБ ЦЕПИ":
            for option in basket['grouped_options'][0]['options']:
                if 'плетен' in option['name']:
                    lock_type = option['value']
    except:
        pass
    return date, name, competitor, group, weight, insert, lock_type, metal

In [ ]:
def tn (i):
    inss = str(df.iloc[i]['insert']).split(',')
    for ins in inss:
        ins = str(ins).strip().lower()
        insertgroup = ""
        if ins == "бриллиант" or ins == "сапфир" or ins =="топаз" or ins == "изумруд" or ins == "бриллиант натуральный" or ins == "сапфир натуральный" or ins == "топаз натуральный" or ins == "изумруд натуральный" or ins =="бриллианты" or ins == "":
            insertgroup = "ДК"
            tovnap  = 'ДК'
            break
        elif ins == "фианит" or ins == "бриллиант искусственный" or ins == "бриллиант выращенный" or ins =="бриллианит" or ins == "бриллиант синтетический" or "фианит" in str(df.iloc[i]['insert']).lower():
            insertgroup = "ИФ"
            tovnap = "ПДК"
        elif "нет" in ins or "без" in ins or  str(df.iloc[i]['insert']).lower().strip() == "золото" or ins == "nan":
            insertgroup = "БК"
            tovnap  = 'БК'
            break
        elif "бр" in ins and "искусственный" not in ins and "выращенный" not in ins and "синтетический" not in ins and "браслет" not in ins:
            insertgroup = "ДК"
            tovnap = 'ДК'
            break
        elif insertgroup != "ИФ":
            insertgroup = "ПДК"
            tovnap = "ПДК"
    return insertgroup, tovnap

In [ ]:
married = True
parsesize = False
warnings.filterwarnings('ignore')
links=list()
spis = [143, 287, 431, 719, 1007, 1061, 1115, 1169, 1313, 1601, 1655, 1919, 2045, 2189, 2405]
p=1
arts = list()
prices = list()
date = list()
discount = list()

In [ ]:
mwd = webdriver.Chrome()
mwd.maximize_window()
df =pd.DataFrame(columns=['date','name','competitor','group','weight','insert','lock_type','metal','price','price_old','discount','art','avg_price'])
while len(df.index)<150:
    url = f'https://www.wildberries.ru/catalog/0/search.aspx?page={p}&sort=popular&search=%D0%B7%D0%BE%D0%BB%D0%BE%D1%82%D0%BE&f1134=10592'
    mwd.get(url)
    time.sleep(1.5)
    step = 1900
    while True:
        step0=step-1900
        lh = mwd.execute_script("return document.body.scrollHeight")
        mwd.execute_script(f"window.scrollTo({step0}, {step});")
        time.sleep(1)
        nh = mwd.execute_script("return document.body.scrollHeight")
        step+=1900
        if nh==lh:
            break
    soup = bs(mwd.page_source, "html.parser")
    artss =  soup.find_all('article',id=True)
    for art in artss:
        artt = art['data-nm-id'] 
        try:
            rate = art.find_next('span',attrs={'class':'address-rate-mini address-rate-mini--sm'}).text
            prcount = art.find_next('span',attrs={'product-card__count'}).text
        except:
            rate = 1
            prcount = 1
        try:
            prcount = int(prcount[:prcount.find(" ")])
        except:
            prcount = 1
        if float(rate) >=4.7 and int(prcount)>100:
            if str(art).find('price__lower-price wallet-price')>0:
                prl = art.findNext('ins',attrs={'class':'price__lower-price wallet-price'}).text.strip()[:-2].replace('\xa0','')
                po = math.ceil(int(prl)/0.95)
                discount = 0.95
            else:
                prl = art.findNext('ins',attrs={'class': 'price__lower-price'}).text.strip()[:-2]
                if prl.find('\xa0')>0:
                    prl = prl.replace('\xa0','')
                    po = prl
                    discount = 1
            params = card(artt)
            params = list(params)
            if not np.isnan(params[4]) and int(prl)/params[4]>3000 and "жемч" not in str(params[5]) and "кауч" not in str(params[5]) and "эмал" not in str(params[5]):  
                params.append(prl)
                params.append(po)
                params.append(discount)
                params.append(artt)
                params.append(int(prl)/params[4])
                df.loc[len(df.index)] = params
                if len(df.index) == 150:
                    break
    p+=1
df = df[['date','competitor','name','group','weight','price','price_old','discount','insert','metal','lock_type','art','avg_price']]
mwd.close()
df

In [ ]:
df['group'].value_counts()

In [63]:
tgs = list()
tnns = list()
for i in df.index:
    if df.iloc[i]['group'] == "Ювелирные подвески":
        tg = "ПОДВЕС " + charms(i)
    elif df.iloc[i]['group'] == "Ювелирные кольца":
        if "обруч" in df.iloc[i]['name'].lower() or "помолв" in df.iloc[i]['name'].lower() or "свад" in df.iloc[i]['name'].lower():
            tg = "КОЛЬЦА ОБРУЧ"
        else:
            tg = "КОЛЬЦА"
    elif df.iloc[i]['group'] == "Ювелирные цепочки":
        tg = "ЦБ ЦЕПИ"
    elif df.iloc[i]['group'] == "Ювелирные браслеты":
        tg = "ЦБ БРАСЛЕТЫ"
    elif df.iloc[i]['group']=="Ювелирные серьги":
        tg = "СЕРЬГИ"
    else:
        tg = ""
    if len(tg)>0 and "ЦБ" not in tg:
        tg = f'{tn(i)[0]} {tg}'.strip()
        tnn = tn(i)[1]
    elif "ЦБ" in tg:
        tnn = "ЦБ"
    else:
        tnn = "Другое"
        tg = "Другое"
    tgs.append(tg)
    tnns.append(tnn)
df.insert(loc=len(df.columns),column='product_group(ТГ)',value=tgs)
df.insert(loc=len(df.columns),column='product_direction(ТН)',value=tnns)

ДК ДК
ДК ДК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
БК БК
БК БК
ПДК ПДК
ПДК ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ДК ДК
ДК ДК
ДК ДК
ДК ДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
ПДК ПДК
ПДК ПДК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ПДК ПДК
ПДК ПДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
ДК ДК
ДК ДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ДК ДК
ДК ДК
ИФ ПДК
ИФ ПДК
ДК ДК
ДК ДК
ДК ДК
ДК ДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
ПДК ПДК
ПДК ПДК
ИФ ПДК
ИФ ПДК
ПДК ПДК
ПДК ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ПДК ПДК
ПДК ПДК
ДК ДК
ДК ДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ПДК ПДК
ПДК ПДК
ДК ДК
ДК ДК
БК БК
БК БК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ДК ДК
ДК ДК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ИФ ПДК
ИФ ПДК
ДК ДК
ДК ДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
ИФ ПДК
БК БК
БК БК
ПДК ПДК
ПДК ПДК
БК БК
БК БК
ПДК ПДК
ПДК ПДК
ПДК ПДК
ПДК ПДК
ИФ

ValueError: cannot insert product_group(ТГ), already exists

In [64]:
df

,date,competitor,name,group,weight,price,price_old,discount,insert,metal,lock_type,art,avg_price,product_group(ТГ),product_direction(ТН)
0,2024-03-20,SOKOLOV,кольцо золотое 585 пробы с натуральным бриллиа...,Ювелирные кольца,1.51,17909,17909,1.00,бриллиант натуральный,золото красное,NaN,7058175,11860.264901,ДК КОЛЬЦА,ДК
1,2024-03-20,REDzoloto,кольцо золотое 585 пробы с фианитами,Ювелирные кольца,1.33,11995,12627,0.95,фианит,золото красное,NaN,35443032,9018.796992,ИФ КОЛЬЦА,ПДК
2,2024-03-20,DIALVI,кольцо золотое 585 спаси и сохрани,Ювелирные кольца,1.50,11805,12427,0.95,без вставки,золото,NaN,91721346,7870.000000,БК КОЛЬЦА,БК
3,2024-03-20,Золотая Подкова,кольцо золотое 585 проба,Ювелирные кольца,1.30,12371,13023,0.95,фианит,золото красное; фианит,NaN,13294037,9516.153846,ИФ КОЛЬЦА,ПДК
4,2024-03-20,Золотая Подкова,кольцо золотое 585 проба,Ювелирные кольца,0.95,6289,6620,0.95,фианит,золото красное,NaN,98618802,6620.000000,ИФ КОЛЬЦА,ПДК
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2024-03-20,FLARE,золотая цепочка на шею 585,Ювелирные цепочки,1.78,9519,10020,0.95,"цепочка золотая на шею , цепь золотая без вста...",золото красное,шпрингельный; карабин,171620906,5347.752809,ЦБ ЦЕПИ,ЦБ
146,2024-03-20,SOKOLOV,серьги золотые 585 пробы с камнями длинные про...,Ювелирные серьги,2.50,21909,21909,1.00,фианит; топаз натуральный,золото красное,без замка,8201360,8763.600000,ИФ СЕРЬГИ,ПДК
147,2024-03-20,KARATOV,золотая подвеска на шею козерог 585 пробы,Ювелирные подвески,0.47,2924,3078,0.95,без вставки,золото красное,NaN,53879384,6221.276596,БК ПОДВЕС ДЕКОР,БК
148,2024-03-20,Золотая Подкова,кольцо золотое 585 проба,Ювелирные кольца,1.07,7355,7743,0.95,фианит,золото красное; фианит,NaN,98825277,6873.831776,ИФ КОЛЬЦА,ПДК


In [65]:
df['product_group(ТГ)'].value_counts()

product_group(ТГ)
ИФ КОЛЬЦА           33
ИФ СЕРЬГИ           21
ДК КОЛЬЦА           13
БК ПОДВЕС ДЕКОР     11
БК СЕРЬГИ            8
ПДК ПОДВЕС ДЕКОР     7
ПДК СЕРЬГИ           7
БК ПОДВЕС КУЛЬТ      7
ИФ ПОДВЕС ДЕКОР      7
БК КОЛЬЦА            7
ЦБ ЦЕПИ              6
ЦБ БРАСЛЕТЫ          5
БК КОЛЬЦА ОБРУЧ      4
ДК СЕРЬГИ            4
Другое               2
ДК ПОДВЕС ДЕКОР      2
ИФ ПОДВЕС КУЛЬТ      2
ДК ПОДВЕС КУЛЬТ      1
ПДК КОЛЬЦА ОБРУЧ     1
ПДК КОЛЬЦА           1
ИФ КОЛЬЦА ОБРУЧ      1
Name: count, dtype: int64

In [66]:
df['product_direction(ТН)'].value_counts()

product_direction(ТН)
ПДК       80
БК        37
ДК        20
ЦБ        11
Другое     2
Name: count, dtype: int64

In [68]:
df.to_excel(f'wildberries(top)_{datetime.date.today().day}_{datetime.date.today().month}_{datetime.date.today().year}.xlsx',index=False)